In [1]:
# Import des packages

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

import time
import pandas as pd
import os

In [2]:
# URL de la page souhaitée

url = "https://www.sustainalytics.com/esg-ratings"

# Où seront stockée les données
output_file = "real_scraped_data.csv"


In [ ]:
# Fonction qui sauvegarde les données au fur et à mesure du scraping dans un DF

def save_data(data, file_path):
    """  
    Cette fonction a pour but de sauvegarder les données dans un fichier CSV au fur et à mesure du scraping
    Ainsi, compte tenu du fait qu'il prend du temps à être exécuté, on évite de perdre les données déjà collectées
    
    """
    df = pd.DataFrame(data)
    if os.path.exists(file_path):
        df.to_csv(file_path, index=False, mode='a', header=False)
    else:
        df.to_csv(file_path, index=False)


def scrape_company_details(driver, wait):
    """  
    Cette fonction a pour but de récupérer les détails de chaque entreprise : note ESG, pays, date de la dernière mise à jour, secteur et nombre d'employés
    """
    try:
        grade = wait.until(EC.presence_of_element_located(
            (By.CLASS_NAME, "risk-rating-score"))).text
        
        country = driver.find_element(By.CLASS_NAME, "country").text

        try:
            date = driver.find_element(By.XPATH, "//span[contains(text(), 'Last Full Update')]/strong").text
        except Exception:
            date = "Non disponible"

        try:
            industry = driver.find_element(By.CSS_SELECTOR, "strong.industry-group").text
        except Exception:
            industry = "Non disponible"

        try:
            employees = driver.find_element(By.XPATH, "//div[contains(text(), 'Full time employees')]/strong").text
        except Exception:
            employees = "Non disponible"

        return {"grade": grade, "country": country, "date": date, "industry": industry, "employees": employees}
    except (TimeoutException, StaleElementReferenceException) as e:
        print(f"Erreur détails: {e}")
        return None

def scrape_companies(url, num_pages):
    """  
    Cette fonction a pour but de scraper les données de toutes les entreprises sur la page dont il est question
    
    """
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)
    data = [] # On initialise une liste vide pour stocker les données
    
    try:
        driver.get(url)
        
        for page in range(num_pages):
            print(f"Scraping page {page + 1}")
            
            # On attend que les données nécessaire soient bien chargées pour passer à la suite
            company_links = wait.until(EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "a.primary-color.d-block.js-fix-path")))

            companies = []
            for link in company_links:
                try:
                    name = link.text
                    href = link.get_attribute('href') # On récupère le lien sur lequel il faudra ensuite cliquer
                    # pour accéder aux détails de l'entreprise
                    companies.append((name, href)) # On ajoute tout les noms de firme de la même page dans une liste
                except StaleElementReferenceException:
                    print("Lien obsolète, ignoré.")

            
            for name, company_url in companies:
                try:
                    # Pour chaque firme on ouvre un nouvel onglet
                    driver.execute_script("window.open(arguments[0]);", company_url)
                    # Passer à la nouvelle fenêtre
                    driver.switch_to.window(driver.window_handles[1])
                    
                    # On récupère ici la fonction qui nous permet d'obtenir les détails
                    details = scrape_company_details(driver, wait)
                    if details:
                        record = {
                            "name": name,
                            "grade": details["grade"],
                            "country": details["country"],
                            "date": details["date"],
                            "industry": details["industry"],
                            "employees": details["employees"]
                        }
                        data.append(record)
                    
                        save_data([record], output_file)

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    
                except Exception as e:
                    print(f"Erreur entreprise {name}: {e}")
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    continue
            
            try:
                next_page = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, f"a.pagination-page[href='hrefcurrentpage={page+2}']"))
                )
                # On oublie pas mettre le "+2" juste au-dessus, puisqu'on commence avec page = 0 sur la page 1
                # alors pour arriver à la page suivane qui est la 2ème, on doit ajouter 2

                # La commande suivante simule le clique sur le lien de la page suivante
                driver.execute_script("arguments[0].click();", next_page)
                time.sleep(3)
            except Exception as e:
                print(f"Pas de page suivante ou erreur: {e}")
                break
            except StaleElementReferenceException:
                print("Élément obsolète. Nouvelle tentative...")
                continue


    finally:
        driver.quit() # Une fois le scraping terminé, on ferme le navigateur
    return pd.DataFrame(data)

df = scrape_companies(url,  1391)
print(df)
